<a href="https://colab.research.google.com/github/yoursdream123/PIAIC-Q2-PROJECTS/blob/main/RAG_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain-pinecone

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_2Zm5Ym_PttGgRU6SikpLfXQAqvAvDs2TiQpPBLKoUH8sE8qFPyaySpxeteC3LGDesCSopc")

In [12]:
index_name = "rag-system-project"

pc.create_index(
    name=index_name,
    dimension=768, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [13]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00


In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("Pakistan Zindabad!")
vector[:5]

[0.00918805506080389,
 -0.03816039487719536,
 0.03429127112030983,
 -0.04830525070428848,
 0.02473844774067402]

In [15]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index="rag-system-project", embedding=embeddings)

In [16]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata = {"source": "tweet"},
)
document_1

Document(metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')

In [17]:
#Data save
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

len(documents)

10

In [26]:
from langchain_pinecone import PineconeVectorStore
# Assuming 'pc' is your Pinecone client and 'index_name' is the name of your index
vector_store = PineconeVectorStore(index=pc.Index(index_name), embedding=embeddings)

In [27]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['68ed621a-5846-4d20-bb8f-f4bb47ede8d6',
 'a415c89d-6865-4286-98b9-8aeca16b13f5',
 '9aa206e8-aea6-45b7-8718-b70070138d25',
 '5485f655-25b5-43f9-8fb0-fb6d43516ba1',
 'bd568b6b-3686-476f-bb21-dc43d551f3c0',
 'ffd8b120-10e6-4273-a3e6-681d7584b727',
 '56bbde6b-6b31-46a2-ab14-27f521c9179c',
 '22acf113-8bf3-4867-ad73-a815e7909a33',
 '29f6e839-6790-4348-8252-554e8b505e25',
 '09feb08f-f592-4c50-98b0-fc31c3c27a42']

In [28]:
#Data retrieve
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [29]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?",
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.577374] I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* [SIM=0.537337] I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]
* [SIM=0.533720] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]


In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [31]:
def answer_to_user(query:str):
  #vector search
  vector_results = vector_store.similarity_search(query, k=2)

#passing to model user query and vector results
  final_answer = llm.invoke(f"ANSWER THE USER QUERY: {query}, Here are some reference to answer {vector_results}")

  return final_answer

In [32]:
answer = answer_to_user("LangChain provides abstractions to make working with LLMs easy")
answer.content

"Based on the provided documents, LangChain is a framework used for building applications, as evidenced by the mention of a new project being built with it.  One document highlights a different framework, LangGraph, but doesn't directly compare or contrast it with LangChain's ease of use with LLMs.  Therefore, the provided text only confirms that LangChain is used for application building, not specifically its ease of use with LLMs.  More information is needed to answer the query fully.\n"